# 偵測和分析臉部

電腦視覺解決方案通常需要人工智慧 (AI) 解決方案，以便能夠偵測、分析或識別人類臉部。例如，假設零售公司 Northwind Traders 已經決定實作「智慧商店」，在智慧商店中 AI 服務監視商店，以便識別需要幫助的客戶，並指揮員工去幫助他們。實現此目標的一種方式是執行臉部偵測和分析，換言之，即判斷影像中是否出現任何臉部，若有，則分析其特徵。

![一個傀儡程式正在分析臉部](./images/face_analysis.jpg)

## 使用臉部認知服務來偵測臉部

假設 Northwind Traders 想要建立的智慧商店系統需要能夠偵測客戶並分析其臉部特徵。在 Microsoft Azure 中，您可以使用**臉部** (Azure 認知服務的組成部分) 來達成此目標。

### 建立認知服務資源

讓我們首先在您的 Azure 訂用帳戶中建立**認知服務**資源。

> **備註**：若您已經有認知服務資源，只需在 Azure 入口網站中開啟其 [**快速入門**] 頁面並將其金鑰和端點複製到下面的儲存格。否則，可追隨下面的步驟來建立一個認知服務資源。

1. 在其它瀏覽器索引標籤中，透過 https://portal.azure.com 開啟 Azure 入口網站，用您的 Microsoft 帳戶登入。
2. 按一下 [**&#65291; 建立資源**] 按鈕，搜尋*認知服務*， 並建立包含以下設定的**認知服務**資源：
    - **訂用帳戶**：*您的 Azure 訂用帳戶*。
    - **資源群組**：*選取或建立具有唯一名稱的資源群組*。
    - **區域**：*選擇任一可用區域*：
    - **名稱**：*輸入唯一名稱*。
    - **定價層**：S0
    - **我確認已閱讀通知並理解通知內容**：已選取。
3. 等待部署完成。然後前往您的認知服務資源，在 [**概觀**] 頁面上，按一下連結以管理服務金鑰。您將需要端點和金鑰，以便從用戶端應用程式連線到您的認知服務資源。

### 獲取您的認知服務資源之金鑰和端點

若要使用您的認知服務資源，用戶端應用程式需要其端點和驗證金鑰：

1. 在 Azure 入口網站中，您的認知服務資源之 [**金鑰和端點**] 頁面上，複製您的資源之**金鑰 1** 並將其貼上到下面的程式碼，取代 **YOUR_COG_KEY**。

2. 複製您的資源之**端點**並將其貼上到下面的程式碼，代替 **YOUR_COG_ENDPOINT**。

3. 透過按一下 [執行儲存格] <span>&#9655;</span> 按鈕 (位於儲存格左上方) 執行下面的儲存格中的程式碼。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

現在您擁有認知服務資源，可以使用臉部服務來偵測商店中的人類臉部。

執行下面的程式碼儲存格以便查看範例。

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces)

唯一的識別碼已指派給每張已偵測到的臉部，因此您的應用程式可以識別已偵測到的每一張個人臉部身分。

執行下面的儲存格以便查看更多顧客臉部的識別碼。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

## 分析臉部屬性

臉部不只是能簡單地偵測臉部，還有更多功能。臉部亦可分析臉部特徵和表情來提示年齡和情緒狀態；例如，執行下面的程式碼來分析顧客的臉部屬性。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# Detect faces and specified facial attributes
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Display the faces and attributes (code in python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

根據影像中為客戶偵測到的情緒分數，客戶似乎對購物體驗頗為滿意。

## 尋找相似的臉部 

臉部識別碼 (為每張已偵測到的臉部而建立) 用於單獨識別臉部偵測。您可以使用這些識別碼將已偵測到的臉部與之前已偵測到的臉部進行比較，並尋找具有相似特徵的臉部。

例如，執行下面的儲存格來比較一張影像和另一張影像中的顧客，並尋找相符的臉部。

In [ ]:
# Get the ID of the first face in image 1
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Get the face IDs in a second image
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Find faces in image 2 that are similar to the one in image 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Show the face in image 1, and similar faces in image 2(code in python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## 識別臉部

到目前為止，您已經看到臉部可以偵測臉部和臉部特徵，還可以識別彼此相似的兩張臉部身分。您可以透過實作臉部識別解決方案更進一步，在該解決方案中訓練臉部以便識別特定的人臉。該解決方案在各種案例中均實用，例如在社交媒體應用程式中自動標記朋友的相片，或使用臉部識別作為生物特徵身分識別驗證系統的一部分。

為了了解其工作原理，我們假設 Northwind Traders 公司想要使用臉部識別來確保只有 IT 部門中獲得授權的員工可以存取安全系統。

我們首先建立一個*人員群組*來代表獲得授權的員工。

In [ ]:
group_id = 'employee_group_id'
try:
    # Delete group if it already exists
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

現在已有*人員群組*，我們可以為想要納入群組的每位員工新增一個*人員*，然後註冊每個人的多張相片以便臉部可以了解每個人的相異臉部特性。理想情況下，影像應該顯示同一個人，這個人呈現不同造型和不同的臉部表情。

我們將新增一位名叫 Wendell 的員工，並註冊這名員工的三張相片。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Add a person (Wendell) to the group
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Get photo's of Wendell
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Add each photo to person in person group
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Display each image
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

透過已新增的人員和已註冊的相片，我們現在可以訓練臉部來識別每個人。

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

現在，透過已訓練的模型，您可以用它來識別影像中已識別的臉部身分。

In [ ]:
# Get the face IDs in a second image
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Get recognized face names
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# show recognized faces
faces.show_recognized_faces(image_path, image_faces, face_names)



## 深入瞭解

若要深入了解臉部認知服務，請查看[臉部文件](https://docs.microsoft.com/azure/cognitive-services/face/)
